In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing

# Importing and spliting  data

* **20% data split**

In [ ]:
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
df['Id'] = df['Id'].apply(lambda x: '../input/petfinder-pawpularity-score/train/'+ x +'.jpg')
test_df['Id'] = test_df['Id'].apply(lambda x: '../input/petfinder-pawpularity-score/test/'+ x +'.jpg')

In [ ]:
# scaler = StandardScaler()

# scaled_X = scaler.fit_transform(X)
# scaled_X

# EDA

In [ ]:
df['Pawpularity'].hist(figsize = (10, 5))
print(f"The mean Pawpularity score is {df['Pawpularity'].mean()}")
print(f"The median Pawpularity score is {df['Pawpularity'].median()}")
print(f"The standard deviation of the Pawpularity score is {df['Pawpularity'].std()}")

In [ ]:
print(f"There are {len(df['Pawpularity'].unique())} unique values of Pawpularity score")

In [ ]:
df['norm_score'] = df['Pawpularity']/100
df['norm_score']

In [ ]:
df = df.sample(frac=1).reset_index(drop=True) #shuffle dataframe
df.head()

## Finding optimial bin formula

In [ ]:
import math
#Rice rule
num_bins = int(np.ceil(2*((len(df))**(1./3))))
num_bins

In [ ]:
df['bins'] = pd.cut(df['norm_score'], bins=num_bins, labels=False)
df['bins'].hist()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

df['fold'] = -1


N_FOLDS = 10
strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=365, shuffle=True)
for i, (_, train_index) in enumerate(strat_kfold.split(df.index, df['bins'])):
    df.iloc[train_index, -1] = i
    
df['fold'] = df['fold'].astype('int')

df.fold.value_counts().plot.bar()

In [ ]:
df['bins'].value_counts()

In [ ]:
df

In [ ]:
X = df.drop(['Id', "Pawpularity", "fold", "norm_score", "bins"], axis=1)
y = df['Pawpularity']

X.shape

## Changing Test Data

# Creating Input Datasets

* **Funtion creat_dataset_metadata --> creates dataset of images and metadata**
* **Function creat_dataset_image --> creates dataset with only images**
* **Function img_read --> reads image from image folder and resize it to (229, 229)**

In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 64
img_path_train = '../input/petfinder-pawpularity-score/train/'
img_path_test = '../input/petfinder-pawpularity-score/test'

In [ ]:
def img_read(is_labelled):
    def read_img(image_path):
        img = tf.io.read_file(image_path)
        img = tf.io.decode_jpeg(img, channels=3)
        img = tf.cast(img, tf.float32)
        # normalizing image by calculating (img - mean) / adjusted_std
        # https://www.tensorflow.org/api_docs/python/tf/image/per_image_standardization
#         img = tf.image.per_image_standardization(img)
        img = tf.image.resize(img, IMG_SIZE)
        return img
    
    def can_be_readed(path, label):
        return read_img(path), label

    return can_be_readed if is_labelled else read_img

In [ ]:
def creat_dataset_metadata(df, batch_size, is_labelled = True):
    # function to convert images 
    image_read = img_read(is_labelled)
    
    # creating dataset of image path and pawpularity score
    if is_labelled:
        meta_data = df.drop(['Id', "Pawpularity", "fold", "norm_score", "bins"], axis=1)
#         scaled_meta_data = scaler.transform(meta_data)
        
        input_dataset = tf.data.Dataset.from_tensor_slices((df["Id"].values, meta_data))
        output_dataset = tf.data.Dataset.from_tensor_slices((df["Pawpularity"].values))
        
        # converting images to tensors
        input_dataset = input_dataset.map(image_read, num_parallel_calls=tf.data.AUTOTUNE)
        
        # creating final dataset
        dataset = tf.data.Dataset.zip((input_dataset, output_dataset))
        
        # spliting in batches
        dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset
        
    else :
        meta_data = df.drop('Id', axis=1)
#         saled_meta_data = scaler.transform(meta_data)
        
        input_dataset_1 = tf.data.Dataset.from_tensor_slices((df["Id"].values))
        input_dataset_2 = tf.data.Dataset.from_tensor_slices((meta_data))
        
        # converting images to tensors
        input_dataset_1 = input_dataset_1.map(image_read, num_parallel_calls=tf.data.AUTOTUNE)
        
#         dataset = tf.data.Dataset.from_tensor_slices((input_dataset_1, input_dataset_2))
        dataset = tf.data.Dataset.zip(((input_dataset_1, input_dataset_2),)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        return dataset

In [ ]:
def creat_dataset_image(df, batch_size, is_labelled = True):
    # function to convert images 
    image_read = img_read(is_labelled)
    
    # creating dataset of image path and pawpularity score
    if is_labelled:
        input_dataset = tf.data.Dataset.from_tensor_slices((df["Id"].values, df["Pawpularity"].values))
        
        # converting images to tensors
        dataset = input_dataset.map(image_read, num_parallel_calls=tf.data.AUTOTUNE)
        
        # spliting in batches
        dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        return dataset
        
    else :
        input_dataset_1 = tf.data.Dataset.from_tensor_slices((df["Id"].values))
        
        # converting images to tensors
        input_dataset_1 = input_dataset_1.map(image_read, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = input_dataset_1.batch(batch_size).prefetch(tf.data.AUTOTUNE)
        
        return dataset

## Datasets for model.2

In [ ]:
test_dataset_metadata = creat_dataset_metadata(test_df, BATCH_SIZE, False)

In [ ]:
test_dataset_metadata

## Data set form images only model

In [ ]:
test_dataset_image = creat_dataset_image(test_df, BATCH_SIZE, False)

In [ ]:
test_dataset_image

# Augmentation layer and Model

* **For base model we are using inception v3 with pretrained weights and unfreezing last 28 layers**
* **create_model_1 --> creates model for only image inputs**
* **create_model_2 --> reates model for image and metadata**

In [ ]:
# data augmentation stage with horizontal flipping, rotation, zooms, etc....
image_augmentation = tf.keras.Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomRotation(0.3),
    preprocessing.RandomZoom(0.3),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomContrast(0.2),
    preprocessing.Rescaling(1./255)
], name='data_augmentation')

In [ ]:
from tensorflow.keras import layers

xception_weights = '../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
inception_v3 = '../input/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
efficientnet_b2 = '../input/d/aeryss/keras-pretrained-models/EfficientNetB2_NoTop_ImageNet.h5'
base_model = tf.keras.applications.InceptionV3(include_top=False, weights=inception_v3)
base_model.trainable = True

In [ ]:
for layer in base_model.layers[:-28]:
    layer.trainable = False

In [ ]:
def create_model_1():
    # image input model
    img_input = tf.keras.layers.Input(shape=(229, 229, 3), name='image_input')
    x = image_augmentation(img_input)
    x = base_model(x)
    x = GlobalMaxPooling2D()(x)
    x = layers.Dense(526, activation='relu')(x)
    output_layer = layers.Dense(1, activation='linear')(x)

    model = tf.keras.Model(inputs = img_input, outputs=output_layer)
    
    return model

In [ ]:
def create_model_2():
    # image input model
    img_input = tf.keras.layers.Input(shape=(224, 224, 3), name='image_input')
    x = image_augmentation(img_input)
    x = base_model(x)
    x = GlobalMaxPooling2D()(x)
    img_output = layers.Dense(526, activation='relu')(x)
    img_model = tf.keras.Model(img_input, img_output)
 
    # other data Model
    data_input = layers.Input(shape=X.shape[1], name='data_input')
#     x = layers.Dense(64, activation='relu')(data_input)
#     x = layers.Dropout(0.5)(x)
#     x = layers.BatchNormalization()(x)
    data_output = layers.Dense(32, activation='relu')(data_input)
    data_model = tf.keras.Model(data_input, data_output)
    
    # concatinating Model layers
    concat_layer = layers.Concatenate(name = 'concat_layer')([img_model.output, data_model.output])

    combined_dropout = layers.Dropout(0.5)(concat_layer)
    combined_dence = layers.Dense(128, activation='relu')(combined_dropout)
    final_dropout = layers.Dropout(0.5)(combined_dence)

    output_layer = layers.Dense(1, activation='relu')(final_dropout)

    model = tf.keras.Model(inputs = [img_model.input, data_model.input], outputs=output_layer)
    
    return model

In [ ]:
test_model = create_model_2()
test_model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(test_model, show_shapes=True)

# Training callbacks

In [ ]:
train_callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5,
        patience=2, verbose=1
    ),
]

# Model Training

In [ ]:
# list to store outputs of each fold
final_results = []

In [ ]:
for fold in range(10):
    print(f'Fold {fold+1}')
    train_df =  df[df.fold != fold].reset_index(drop=True)
    val_df = df[df.fold == fold].reset_index(drop=True)
    
    # ccreating train and validation dataset
    train_dataset = creat_dataset_metadata(train_df, BATCH_SIZE, is_labelled=True)
    val_dataset = creat_dataset_metadata(val_df, BATCH_SIZE, is_labelled=True)
    
    # creating model
    model = create_model_2()
    
    # compiling model
    model.compile(
        loss = 'mse',
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
        metrics = [tf.keras.metrics.RootMeanSquaredError()],
    )
    
    # fitting Model
    print('model training \n')
    model.fit(
        train_dataset,
        epochs = 25,
        steps_per_epoch = len(train_dataset),
        validation_data = val_dataset,
        validation_steps = len(val_dataset),
        callbacks = train_callbacks,
    )
    
    # making predictions and storing them in list
    submit = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
    pred = model.predict(test_dataset_metadata)
    pred = np.squeeze(pred, axis=1)

    final = pd.DataFrame()
    final['Id'] = submit['Id']
    final['Pawpularity'] = pred
    
    final_results.append(final)
    print(final[:6])

# Calculating final Predictions

In [ ]:
np.array(final_results[:]).shape[0]

In [ ]:
n = np.array(final_results[:]).shape[0]
pred_final = np.dot(np.array([1]*n), np.array(final_results[:])[:,:,1] )
pred_final /= n
pred_final

In [ ]:
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

submission = pd.DataFrame()
submission['Id']= test_df['Id']
submission['Pawpularity']=  (pred_final) + 7
submission.to_csv('submission.csv', index=False)
submission